<a href="https://colab.research.google.com/github/Bill-Roa/Descargar-ERA5-Colab-Hidrologia/blob/main/ERA5_William.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Autor: Santiago Vega Guacaneme
# Fecha: 09/06/2024
# Descripción: Herramienta que descarga y visualiza variables oceanográficas del modelo de clima global ERA5. Descripción del modelo: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview
!pip install cdsapi
!pip install netCDF4
!pip install python-dotenv

import tkinter as tk
from tkinter import ttk
import cdsapi
import datetime
from netCDF4 import Dataset, num2date
import pandas as pd
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"


CoordenadaX = float(input("Coordenada en formato longitud (x): "))
CoordenadaY = float(input("Coordenada en formato latitud (y): "))
Año1 = int(input("Año inicial: "))
Año2 = int(input("Año final: "))
Variable = int(input("Variables: 1. Wind 2. Wave 3. Surface net solar radiation 4. 2m temperature"))

# Puntos con datos más cercanos a las coordenadas dadas
# (X1, Y1) (X2, Y1)
# (X1, Y2) (X2, Y2)
# Puntos = [Y1, X1, Y2, X2]
Puntos = [(int(CoordenadaY / 0.25) + 1) * 0.25, (int(CoordenadaX / 0.25) - 1) * 0.25, (int(CoordenadaY / 0.25)) * 0.25, (int(CoordenadaX / 0.25)) * 0.25]

# Distancia a los puntos con datos más cercanos a las coordenadas dadas
# (d1) (d2)
# (d3) (d4)
# Distancias = [d1, d2, d3, d4]
Distancias = [((CoordenadaX - Puntos[1]) ** 2 + (CoordenadaY - Puntos[0]) ** 2) ** 0.5, ((CoordenadaX - Puntos[3]) ** 2 + (CoordenadaY - Puntos[0]) ** 2) ** 0.5,
              ((CoordenadaX - Puntos[1]) ** 2 + (CoordenadaY - Puntos[2]) ** 2) ** 0.5, ((CoordenadaX - Puntos[3]) ** 2 + (CoordenadaY - Puntos[2]) ** 2) ** 0.5]

# Calculo del factor λ asociado a los puntos con datos más cercanos a las coordenadas dadas (Método de interpolación IDW)
# (λ1) (λ2)
# (λ3) (λ4)
# λ = [λ1, λ2, λ3, λ4]
λ = [(1/a**2)/sum([1/b**2 for b in Distancias]) for a in Distancias]

# Credenciales para descargar datos del modelo ERA5
c = cdsapi.Client(url='https://cds.climate.copernicus.eu/api', key='e987229d-6303-4076-84e1-a02809d733f5') #274197:eced5f0d-418c-4a51-9c1e-cae922a52a81

for year in range(Año1, Año2+1):
    if Variable == 1:
        c.retrieve('reanalysis-era5-single-levels',
            {
                'variable': ['10m_u_component_of_wind', '10m_v_component_of_wind',],
                'year': str(year),
                'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',],
                'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15',
                        '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31',],
                'time': ['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00',],
                'area': Puntos,
                'data_format': 'netcdf',
                'product_type': 'reanalysis',
            },  r'wind_'+str(year)+'.nc')

    elif Variable == 2:
        c.retrieve('reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'variable': ['mean_wave_direction', 'mean_wave_period', 'significant_height_of_combined_wind_waves_and_swell',],
                'year': str(year),
                'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',],
                'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15',
                        '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31',],
                'time': ['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00',],
                'area': [Puntos], #quitar corchetes
                'data_format': 'netcdf',
            },  r'wave_'+str(year)+'.nc')

    elif Variable == 3:
        c.retrieve('reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'variable': ['surface_net_solar_radiation'],
                'year': str(year),
                'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',],
                'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15',
                        '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31',],
                'time': ['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00',],
                'area': [Puntos], #quitar corchetes
                'data_format': 'netcdf',
            },  r'wave_'+str(year)+'.nc')

    elif Variable == 4:
        c.retrieve('reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'variable': ['2m_temperature'],
                'year': str(year),
                'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',],
                'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15',
                        '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31',],
                'time': ['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00',],
                'area': [Puntos], #quitar corchetes
                'data_format': 'netcdf',
            },  r'wave_'+str(year)+'.nc')

data = pd.DataFrame()
for year in range(Año1, Año2+1):
    if Variable == 1:
        file_in = Dataset(r'wind_'+str(year)+'.nc',format='NETCDF4')

        nctime = file_in.variables['valid_time'][:] # get values
        t_unit = file_in.variables['valid_time'].units # get unit  'days since 1950-01-01T00:00:00Z'
        t_cal = file_in.variables['valid_time'].calendar
        tvalue = num2date(nctime,units = t_unit,calendar = t_cal)
        str_time = [i.strftime('%Y-%m-%d %H:%M') for i in tvalue] # to display dates as string
        time = pd.DataFrame(str_time,columns=['Hora'])

        v10_data = file_in.variables['v10'][:]
        u10_data = file_in.variables['u10'][:]
        v10_1 = [v10_data[a,0,0]*λ[0]+v10_data[a,0,0]*λ[1]+v10_data[a,0,0]*λ[2]+v10_data[a,0,0]*λ[3] for a in range(len(v10_data))]
        u10_1 = [u10_data[a,0,0]*λ[0]+u10_data[a,0,0]*λ[1]+u10_data[a,0,0]*λ[2]+u10_data[a,0,0]*λ[3] for a in range(len(u10_data))]
        u10 = pd.DataFrame(u10_1,columns=['u10'])
        v10 = pd.DataFrame(v10_1,columns=['v10'])

        x = pd.concat([time, u10, v10], axis=1)
        data = pd.concat([data, x])

    elif Variable == 2:
        file_in = Dataset(r'wave_'+str(year)+'.nc',format='NETCDF4')

        nctime = file_in.variables['valid_time'][:] # get values
        t_unit = file_in.variables['valid_time'].units # get unit  'days since 1950-01-01T00:00:00Z'
        t_cal = file_in.variables['valid_time'].calendar
        tvalue = num2date(nctime,units = t_unit,calendar = t_cal)
        str_time = [i.strftime('%Y/%m/%d %H:%M') for i in tvalue] # to display dates as string
        time = pd.DataFrame(str_time,columns=['Hora'])

        mwd_data = file_in.variables['mwd'][:]
        mwp_data = file_in.variables['mwp'][:]
        swh_data = file_in.variables['swh'][:]
        mwd_1 = [mwd_data[a,0,0]*λ[0]+mwd_data[a,0,0]*λ[1]+mwd_data[a,0,0]*λ[2]+mwd_data[a,0,0]*λ[3] for a in range(len(mwd_data))]
        mwp_1 = [mwp_data[a,0,0]*λ[0]+mwp_data[a,0,0]*λ[1]+mwp_data[a,0,0]*λ[2]+mwp_data[a,0,0]*λ[3] for a in range(len(mwp_data))]
        swh_1 = [swh_data[a,0,0]*λ[0]+swh_data[a,0,0]*λ[1]+swh_data[a,0,0]*λ[2]+swh_data[a,0,0]*λ[3] for a in range(len(swh_data))]
        mwd = pd.DataFrame(mwd_1,columns=['Mean wave direction'])
        mwp = pd.DataFrame(mwp_1,columns=['Mean wave period'])
        swh = pd.DataFrame(swh_1,columns=['Significant height of combined wind waves and swell'])

        x = pd.concat([time, mwd, mwp, swh], axis=1)
        data = pd.concat([data, x])

    elif Variable == 3:
        file_in = Dataset(r'Surface_net_solar_radiation_'+str(year)+'.nc',format='NETCDF4')

        nctime = file_in.variables['valid_time'][:] # get values
        t_unit = file_in.variables['valid_time'].units # get unit  'days since 1950-01-01T00:00:00Z'
        t_cal = file_in.variables['valid_time'].calendar
        tvalue = num2date(nctime,units = t_unit,calendar = t_cal)
        str_time = [i.strftime('%Y/%m/%d %H:%M') for i in tvalue] # to display dates as string
        time = pd.DataFrame(str_time,columns=['Hora'])

        ssr_data = file_in.variables['ssr'][:]
        ssr_1 = [ssr_data[a,0,0]*λ[0]+ssr_data[a,0,0]*λ[1]+ssr_data[a,0,0]*λ[2]+ssr_data[a,0,0]*λ[3] for a in range(len(ssr_data))]
        ssr = pd.DataFrame(ssr_1,columns=['Surface net solar radiation'])
        x = pd.concat([time, ssr], axis=1)
        data = pd.concat([data, x])

 elif Variable == 4:
        file_in = Dataset(r'2m_temperature_'+str(year)+'.nc',format='NETCDF4')

        nctime = file_in.variables['valid_time'][:] # get values
        t_unit = file_in.variables['valid_time'].units # get unit  'days since 1950-01-01T00:00:00Z'
        t_cal = file_in.variables['valid_time'].calendar
        tvalue = num2date(nctime,units = t_unit,calendar = t_cal)
        str_time = [i.strftime('%Y/%m/%d %H:%M') for i in tvalue] # to display dates as string
        time = pd.DataFrame(str_time,columns=['Hora'])

        2T_data = file_in.variables['2T'][:]
        2T_1 = [2T_data[a,0,0]*λ[0]+2T_data[a,0,0]*λ[1]+2T_data[a,0,0]*λ[2]+2T_data[a,0,0]*λ[3] for a in range(len(2T_data))]
        2T = pd.DataFrame(2T_1,columns=['Surface net solar radiation'])
        x = pd.concat([time, 2T], axis=1)
        data = pd.concat([data, x])


datatoexcel = pd.ExcelWriter(r'Serie de datos '+str(year)+'.xlsx')
data.to_excel(datatoexcel)
datatoexcel._save()

print('DataFrame is written to Excel File successfully.')

from google.colab import files
files.download('Serie de datos '+str(year)+'.xlsx')

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 168)